In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy import stats
import os

In [52]:
pathFiles = '../../Files/SingleFactorExperiment/SHORT/'

scenarios = ['NO', 'ALL', 'ED', 'RAD','LAB', 'OR', 'ICU', 'INP']

In [47]:
def calculate_ed_throughput(df_combined, pathFiles, scenarios):
    """
    Calculates the ED Throughput using the DatasetEDThroughputCorrect and updates df_combined.
    """
    for scenario in scenarios:
        # Construct the file path
        file_path = os.path.join(pathFiles, f'{scenario}/Datasets/DatasetThroughputEDCorrect.csv')
        
        # Check if the file exists
        if not os.path.exists(file_path):
            print(f"File not found for scenario {scenario}: {file_path}")
            continue
        
        # Read the specific dataset for each scenario
        df_ed_throughput = pd.read_csv(file_path, sep=",")
        
        # Set the first column as the index and reset the index
        df_ed_throughput = df_ed_throughput.set_index(df_ed_throughput.columns[0])
        df_ed_throughput = df_ed_throughput.reset_index(drop=True)
        
        # Calculate the mean ED Throughput for each day
        mean_ed_throughput = df_ed_throughput.mean(axis=1)
        
        # Update df_combined with the new ED Throughput values for the specific scenario
        df_combined.loc[df_combined['Scenario'] == scenario, 'ed_throughput_per_day'] = mean_ed_throughput.values

    return df_combined

def recalculate_mortality_rate(df_combined, pathFiles, departments):
    for dept in departments:
        # Read specific datasets for each department
        df_overall = read_specific_dataset(pathFiles, 'DatasetOverallTroughput', dept)
        df_deceased = read_specific_dataset(pathFiles, 'DatasetDeceasedPatients', dept)
        df_doctor_utilization = read_specific_dataset(pathFiles, 'DatasetEDDoctorUtilization', dept)


        # Calculate mortality rate
        mortality_rate = df_deceased['SUM'] * 100 / df_overall['SUM']
        # Update df_combined with new mortality rates for the specific department
        df_combined.loc[df_combined['Scenario'] == dept, 'mortality_rate'] = mortality_rate.values
        df_combined.loc[df_combined['Scenario'] == dept, 'avg_doctor_utilization'] = df_doctor_utilization['Mean'].values


    return df_combined

def read_specific_dataset(path, dataset_name, scenario_label):
    """
    Reads a specific dataset from the "Datasets" folder and returns it as a DataFrame.
    """
    file_path = os.path.join(path, f'{scenario_label}/Datasets/{dataset_name}.csv')
    
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return pd.DataFrame()  # Return an empty DataFrame if the file does not exist
    
    df = pd.read_csv(file_path, sep=",")  # Read the CSV file
    df = df.set_index(df.columns[0])  # Set the first column as the index
    df = df.reset_index(drop=True)  # Reset the index to remove the old index

    keywords = ['Utilization', 'Utiliaztion', 'Nurse', 'Utilaztion', 'Doctor']
    if any(keyword in dataset_name for keyword in keywords):
        df = df * 100
    
    # Calculate SUM and Mean before adding them as columns
    sum_values = df.sum(axis=1)
    mean_values = df.mean(axis=1)


    # Add additional columns
    df['SUM'] = sum_values
    df['Mean'] = mean_values
    df['Scenario'] = scenario_label
    df['Dataset'] = f'Dataset{dataset_name}.csv'
    return df


def preprocess_data(file_path, department_label):
    df = pd.read_csv(file_path, index_col=False)

    # Check if the column ' total_num_ed_patients' exists and rename it
    if ' total_num_ed_patients' in df.columns:
        df.rename(columns={' total_num_ed_patients': 'total_num_ed_patients'}, inplace=True)

    if ' total_num_patients_entered' in df.columns:
        df.rename(columns={' total_num_patients_entered': 'total_num_patients_entered'}, inplace=True)
    
    # Calculate the rate of patients who left without being seen
    df['Rate_LWBS'] = (df['total_num_lwbs'] * 100) / df['total_num_ed_patients_entered']    
    
    # Multiply all utilization columns by 100
    utilization_columns = [col for col in df.columns if 'utilization' in col.lower()]
    for col in utilization_columns:
        df[col] = df[col] * 100

    # Round the values to 0 decimals since only full patients are possible
    for kpi in [
        'avg_queue_length_ed', 'avg_queue_length_radio', 'avg_queue_length_lab',
        'avg_queue_length_surgery', 'throughput_overall', 'ed_throughput_per_day',
        'postponed_Patients_OR', 'postponed_Patients_RAD', 'rejected_Patients_ED']:
        df[kpi] = df[kpi].round(0)
        
    df['Scenario'] = department_label
    return df


In [ ]:

# Constants
E = 0.05
n = 35000





# Load data
df_base_mainFolder_mainKPIs = pd.read_csv(pathMainFolder + '/MainKPIs/MainKPIs.csv', index_col=False)
df_base_mainFolder_mainKPIs['Rate_LWBS'] = (df_base_mainFolder_mainKPIs['total_num_lwbs'] * 100) / df_base_mainFolder_mainKPIs['total_num_ed_patients']

# List of columns to iterate over
columns_to_iterate = [
    'mortality_rate', 'ed_throughput_per_day',
    'avg_waiting_time_icu', 'avg_inward_utilization', 'avg_icu_utilization',
    'avg_waiting_time_ed', 'avg_total_waiting_time', 'throughput_overall',
    'Rate_LWBS', 'rate_target_time_mts_full',
]

# Calculate the confidence interval for each KPI
for column in columns_to_iterate:
    mean = df_base_mainFolder_mainKPIs[column].mean()
    std = df_base_mainFolder_mainKPIs[column].std()
    margin_of_error = E * mean
    z_alpha = margin_of_error / (std / np.sqrt(n))
    confidence_level = norm.cdf(z_alpha) * 2 - 1
    print(f'Confidence interval for {column} with 5% error margin and 10000 iterations: {confidence_level:.2%}')

In [ ]:
# Define constants
z_alpha = 1.96  # Critical value for 95% confidence
n = 0  # Initialize total number of samplings

# Read the main KPI data
df_base_mainFolder_mainKPIs = pd.read_csv(pathMainFolder + '/MainKPIs/MainKPIs.csv', index_col=False)
df_base_mainFolder_mainKPIs['Rate_LWBS'] = (df_base_mainFolder_mainKPIs['total_num_lwbs'] * 100) / df_base_mainFolder_mainKPIs['total_num_ed_patients']

# List of columns to iterate over
columns_to_iterate = [
    'mortality_rate', 'ed_throughput_per_day',
    'avg_inward_utilization', 'avg_icu_utilization',
    'avg_waiting_time_ed', 'avg_total_waiting_time', 'throughput_overall',
    'Rate_LWBS', 'rate_target_time_mts_full', 'rejected_Patients_ED', 'postponed_Patients_RAD', 'postponed_Patients_OR'
]

# Calculate the number of Monte Carlo samplings for each KPI
for column in columns_to_iterate:
    mean_value = df_base_mainFolder_mainKPIs[column].mean()

    # Check if mean is zero
    if mean_value == 0:
        n += 0
        print(f'Number of Monte Carlo Samplings for {column}: 0')
    else:
        # Calculate the margin of error as 1% of the mean
        E = 0.05 * mean_value
        # Calculate the number of samplings needed
        n_kpi = ((z_alpha * df_base_mainFolder_mainKPIs[column].std()) / E) ** 2
        n += round(n_kpi, 0)
        print(f'Number of Monte Carlo Samplings for {column}: {round(n_kpi, 0)}')

# Calculate and print the average number of samplings across all KPIs
n = n / len(columns_to_iterate)
print(f'Average Number of Monte Carlo Samplings for the base model: {n}')

In [ ]:
############################################################################################################
# Calculation of the minimum amount of Monte Carlo samplings for the base model 
# The error margin is 5% of the mean of the KPIs
# The confidence level is 95%
# At the end, the mean of the number of Monte Carlo samplings is calculated
############################################################################################################
# Calculate the number of Monte Carlo samplings for the base model
# The error margin is 5% of the mean of the KPIs
E = 0.05
z_alpha = 1.96
n = 0
pathMainFolder = '/Users/Jeremy/Models/HospitalMaster/Files/FULL/'

df_base_mainFolder_mainKPIs = pd.read_csv(pathMainFolder + '/MainKPIs/MainKPIs.csv')
display(df_base_mainFolder_mainKPIs.columns)
df_base_mainFolder_mainKPIs['Rate_LWBS'] = (df_base_mainFolder_mainKPIs['total_num_lwbs'] * 100) / df_base_mainFolder_mainKPIs['total_num_ed_patients']
# List of columns to iterate over
columns_to_iterate = [
    'mortality_rate', 'ed_throughput_per_day', 'avg_door_to_doctor_time',
    'avg_waiting_time_icu', 'avg_inward_utilization', 'avg_icu_utilization',
    'avg_waiting_time_ed', 'avg_total_waiting_time', 'throughput_overall',
    'Rate_LWBS','rate_target_time_mts_full', 'avg_edwin_score'
]

#columns_to_iterate = [
#    'mortality_rate', 'throughput_overall', 'avg_door_to_doctor_time',
#    'avg_inward_utilization', 'avg_icu_utilization',
#    'avg_total_waiting_time', 'avg_queue_length_ed', 
#]


# Calculate the number of Monte Carlo samplings for each KPI
for column in columns_to_iterate:
    #Check before the calculation if the mean or std is 0
    if df_base_mainFolder_mainKPIs[column].mean() == 0:
        n += 0
        print(f'Number of Monte Carlo Samplings for is {column}: 0')
    else:
        E = df_base_mainFolder_mainKPIs[column].mean() * 0.05
        n += round(((z_alpha * df_base_mainFolder_mainKPIs[column].std()) / (E)) ** 2,0)
        print(f'Number of Monte Carlo Samplings for {column}: {round(((z_alpha * df_base_mainFolder_mainKPIs[column].std()) / (E)) ** 2,0)}')


n = n / len(columns_to_iterate)
print(f'Number of Monte Carlo Samplings for the base model: {n}')

In [ ]:
pathMainFolder = '/Users/Jeremy/Models/HospitalMaster/Files/FULL/'
df_base_mainFolder_mainKPIs = pd.read_csv(pathMainFolder + '/MainKPIs/MainKPIs.csv')
df_base_mainFolder_mainKPIs['Rate_LWBS'] = (df_base_mainFolder_mainKPIs['total_num_lwbs'] * 100) / df_base_mainFolder_mainKPIs['total_num_ed_patients']


In [66]:
def calculate_monte_carlo_samples(df, columns, confidence_level=0.95, relative_error=0.01):
    """
    Calculate required Monte Carlo samples for multiple KPIs
    
    Parameters:
    df (DataFrame): DataFrame with KPI data
    columns (list): List of KPI column names
    confidence_level (float): Desired confidence level (default 0.95)
    relative_error (float): Desired relative error (default 0.05)
    """
    # Z-score for given confidence level
    z_alpha = stats.norm.ppf((1 + confidence_level) / 2)
    print(z_alpha)
    required_samples = {}
    for column in columns:
        mean = df[column].mean()
        std = df[column].std()
        
        if mean == 0 or std == 0:
            print(f"Warning: {column} has mean={mean}, std={std}")
            required_samples[column] = 0
            continue
            
        E = mean * relative_error
        n = ((z_alpha * std) / E) ** 2
        required_samples[column] = int(np.ceil(n))
        
    # Return both detailed and summary results
    return {
        'detailed': required_samples,
        'maximum': max(required_samples.values()),
        'by_kpi': pd.Series(required_samples)
    }

In [63]:
# Combine all MAIN KPIs
df_combined = pd.concat(
    [preprocess_data(f"{pathFiles}/{secnario}/MainKPIs/MainKPIs.csv", secnario) for secnario in scenarios],
    ignore_index=True
)
df_combined = recalculate_mortality_rate(df_combined, pathFiles, scenarios)
df_combined = calculate_ed_throughput(df_combined, pathFiles, scenarios)

In [64]:
df_combined

,total_num_patients,total_num_ed_patients,total_num_patients_entered,total_num_ed_patients_entered,mortality_rate,total_num_deceased_patients,total_num_lwbs,ed_throughput_per_day,avg_queue_length_ed,avg_queue_length_radio,...,avg_queue_length_lab.1,avg_doctor_utilization_surgery,avg_doctor_utilization_radio,avg_xray_utilization,avg_doctor_utilization_lab,avg_nurse_utilization_lab,seed,Rate_LWBS,Scenario,avg_doctor_utilization
0,18883.0,9472.0,19772.0,13328.0,0.993624,191.0,98.0,188.78,7.0,10.0,...,10.275604,74.857346,86.697133,77.847417,30.148664,94.534472,400001.0,0.735294,NO,89.948615
1,18706.0,9432.0,19575.0,13151.0,1.005724,189.0,65.0,188.72,6.0,7.0,...,9.690258,72.817077,85.879516,77.170205,29.928979,94.332597,600001.0,0.494259,NO,88.034825
2,18921.0,9602.0,19789.0,13273.0,1.034538,199.0,146.0,191.30,10.0,13.0,...,11.326395,71.755632,88.361386,82.476118,29.695642,91.420082,1100001.0,1.099977,NO,93.833326
3,18727.0,9426.0,19689.0,13270.0,0.985749,189.0,153.0,187.60,7.0,12.0,...,7.237302,77.682669,85.802405,76.132689,29.248823,91.854588,800001.0,1.152977,NO,88.392766
4,18818.0,9580.0,19737.0,13275.0,0.977424,188.0,71.0,192.06,6.0,8.0,...,8.591174,74.328462,87.850975,80.961667,30.221275,92.202873,100001.0,0.534840,NO,89.823297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195,18607.0,9444.0,19533.0,13174.0,0.955105,180.0,72.0,188.54,7.0,14.0,...,11.019983,69.665459,92.504691,86.331534,29.145933,92.640842,100097.0,0.546531,INP,88.943948
2196,18521.0,9101.0,19656.0,13148.0,1.004725,189.0,34.0,180.80,5.0,11.0,...,9.679434,72.680134,89.894345,81.656332,30.306165,93.105388,100099.0,0.258594,INP,87.651328
2197,18544.0,9161.0,19675.0,13134.0,0.934024,177.0,68.0,182.82,5.0,12.0,...,7.589509,77.198892,89.843159,82.617662,29.213219,92.504487,100098.0,0.517740,INP,85.544473
2198,18699.0,9351.0,19833.0,13279.0,0.923986,177.0,139.0,186.48,8.0,13.0,...,6.899251,79.543270,87.556083,78.997848,28.824659,92.274666,100100.0,1.046766,INP,89.579093


In [70]:
def calculate_monte_carlo_from_combined(df_combined, columns_to_iterate):
    """
    Calculate minimum Monte Carlo samples for each scenario from combined DataFrame
    
    Parameters:
    df_combined (DataFrame): Combined DataFrame containing all scenarios and metrics
    columns_to_iterate (list): List of KPI columns to analyze
    
    Returns:
    dict: Results for each scenario
    """
    scenario_results = {}
    
    print("\nCalculating minimum Monte Carlo samples for each scenario...")
    print("=" * 70)
    
    for scenario in df_combined['Scenario'].unique():
        try:
            # Get data for current scenario
            scenario_df = df_combined[df_combined['Scenario'] == scenario].copy()
            
            # Calculate Monte Carlo samples for this scenario
            results = calculate_monte_carlo_samples(scenario_df, columns_to_iterate)
            scenario_results[scenario] = results
            
            # Print formatted results
            print(f"\nScenario: {scenario}")
            print("-" * 30)
            print(f"Maximum samples needed: {results['maximum']:,}")
            
            # Print KPI-specific results in a table format
            print("\nSamples needed by KPI:")
            print(f"{'KPI':<30} {'Samples':>10}")
            print("-" * 42)
            for kpi, samples in results['by_kpi'].items():
                print(f"{kpi:<30} {int(samples):>10,}")
            
        except Exception as e:
            print(f"\nError processing {scenario}: {str(e)}")
            continue
    
    # Print summary
    print("\nSummary of Required Samples")
    print("=" * 70)
    print(f"{'Scenario':<15} {'Maximum Samples':>15}")
    print("-" * 32)
    
    max_samples = []
    for scenario, results in scenario_results.items():
        max_samples.append(results['maximum'])
        print(f"{scenario:<15} {results['maximum']:>15,}")
    
    if max_samples:
        overall_max = max(max_samples)
        print("\nOverall maximum required samples:", f"{overall_max:,}")
    
    return scenario_results

# Example usage:
columns_to_iterate = [
    'mortality_rate','avg_inward_utilization', 'avg_icu_utilization',
    'avg_waiting_time_ed', 'avg_total_waiting_time', 'throughput_overall',
    'Rate_LWBS', 'rate_target_time_mts_full', 'avg_edwin_score'
]

# Calculate samples using the combined DataFrame
all_results = calculate_monte_carlo_from_combined(df_combined, columns_to_iterate)


Calculating minimum Monte Carlo samples for each scenario...
1.959963984540054

Scenario: NO
------------------------------
Maximum samples needed: 8,215

Samples needed by KPI:
KPI                               Samples
------------------------------------------
mortality_rate                        292
avg_inward_utilization                 23
avg_icu_utilization                   258
avg_waiting_time_ed                   957
avg_total_waiting_time                607
throughput_overall                      2
Rate_LWBS                           8,215
rate_target_time_mts_full           1,201
avg_edwin_score                     1,397
1.959963984540054

Scenario: ALL
------------------------------
Maximum samples needed: 565

Samples needed by KPI:
KPI                               Samples
------------------------------------------
mortality_rate                        234
avg_inward_utilization                 25
avg_icu_utilization                   269
avg_waiting_time_ed            